# 📐 Tree Embedding Protocols — Camera-Ready (Crash-Resilient)

**Paper:** *Dimensional Efficiency Bounds for Embedding Hierarchical Metric Structures*

## Execution Plan

| Step | Protocol | Mode | Time (T4) |
|------|----------|------|----------|
| 1 | E1 | exact | ~10 min |
| 2 | E1 | stochastic | ~25 min |
| 3 | E2 | stochastic | ~30 min |
| 4 | E3 | stochastic | ~15 min |
| **Total** | | | **~80 min** |

⚡ **Crash-Resilient:** All outputs saved to Google Drive. Auto-resume on reconnect.

In [ ]:
# @title 0️⃣ Mount Google Drive + Setup Persistence
from google.colab import drive
import os
import json
from datetime import datetime

# Mount Drive
drive.mount('/content/drive')

# Base directory for all outputs
BASE_DIR = "/content/drive/MyDrive/cgt_tree_runs/"

# Create directories
os.makedirs(BASE_DIR, exist_ok=True)
os.makedirs(f"{BASE_DIR}/checkpoints", exist_ok=True)
os.makedirs(f"{BASE_DIR}/results", exist_ok=True)
os.makedirs(f"{BASE_DIR}/logs", exist_ok=True)

print(f"✅ Google Drive mounted")
print(f"📁 BASE_DIR: {BASE_DIR}")
print(f"📁 Checkpoints: {BASE_DIR}/checkpoints")
print(f"📁 Results: {BASE_DIR}/results")
print(f"📁 Logs: {BASE_DIR}/logs")

Mounted at /content/drive
✅ Google Drive mounted
📁 BASE_DIR: /content/drive/MyDrive/cgt_tree_runs/
📁 Checkpoints: /content/drive/MyDrive/cgt_tree_runs//checkpoints
📁 Results: /content/drive/MyDrive/cgt_tree_runs//results
📁 Logs: /content/drive/MyDrive/cgt_tree_runs//logs


In [ ]:
# @title 0️⃣.1 Progress Tracking Functions

PROGRESS_FILE = f"{BASE_DIR}/progress.txt"

def save_progress(marker: str):
    """Save completed step marker to Drive."""
    completed = load_progress()
    if marker not in completed:
        completed.append(marker)
    with open(PROGRESS_FILE, "w") as f:
        f.write("\n".join(completed))
    print(f"💾 Progress saved: {marker}")

def load_progress() -> list:
    """Load list of completed steps."""
    if os.path.exists(PROGRESS_FILE):
        with open(PROGRESS_FILE, "r") as f:
            return [line.strip() for line in f.readlines() if line.strip()]
    return []

def is_completed(marker: str) -> bool:
    """Check if step was already completed."""
    return marker in load_progress()

def reset_progress():
    """Reset all progress (use with caution)."""
    if os.path.exists(PROGRESS_FILE):
        os.remove(PROGRESS_FILE)
    print("⚠️ Progress reset")

def save_result(name: str, data: dict):
    """Save result to Drive."""
    filepath = f"{BASE_DIR}/results/{name}.json"
    with open(filepath, "w") as f:
        json.dump(data, f, indent=2)
    print(f"💾 Result saved: {filepath}")

def load_result(name: str) -> dict:
    """Load result from Drive."""
    filepath = f"{BASE_DIR}/results/{name}.json"
    if os.path.exists(filepath):
        with open(filepath, "r") as f:
            return json.load(f)
    return None

def log_message(msg: str):
    """Append message to log file."""
    timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    logfile = f"{BASE_DIR}/logs/run_log.txt"
    with open(logfile, "a") as f:
        f.write(f"[{timestamp}] {msg}\n")

# Show current progress
completed = load_progress()
print(f"\n📊 Current progress: {len(completed)} steps completed")
for step in completed:
    print(f"   ✅ {step}")


📊 Current progress: 0 steps completed


In [ ]:
# @title 1️⃣ Install Dependencies
!pip install -q torch numpy scipy ripser
print("✅ Dependencies installed")
log_message("Dependencies installed")

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 842.1/842.1 kB 20.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.6/48.6 kB 4.1 MB/s eta 0:00:00
✅ Dependencies installed


In [ ]:
# @title 2️⃣ Upload Module
from google.colab import files
import shutil

# Check if module already exists in Drive
module_path_drive = f"{BASE_DIR}/tree_protocols_standalone.py"
module_path_local = "/content/tree_protocols_standalone.py"

if os.path.exists(module_path_drive):
    # Copy from Drive to local
    shutil.copy(module_path_drive, module_path_local)
    print(f"✅ Module loaded from Drive: {module_path_drive}")
else:
    # Upload and save to Drive
    print("Upload tree_protocols_standalone.py:")
    uploaded = files.upload()
    # Save copy to Drive for future runs
    if os.path.exists(module_path_local):
        shutil.copy(module_path_local, module_path_drive)
        print(f"💾 Module backed up to Drive: {module_path_drive}")

log_message("Module loaded")

Upload tree_protocols_standalone.py:


Saving tree_protocols_standalone.py to tree_protocols_standalone.py
💾 Module backed up to Drive: /content/drive/MyDrive/cgt_tree_runs//tree_protocols_standalone.py


In [ ]:
# @title 3️⃣ Import + Verify Environment
import sys
sys.path.insert(0, '/content')

from tree_protocols_standalone import (
    run_protocol_e1,
    run_protocol_e2,
    run_protocol_e3,
    get_device,
    clear_gpu,
    EXACT_THRESHOLD,
    K_MAX,
    K_MULTIPLIER
)

import torch
device = get_device()

print("="*60)
print("ENVIRONMENT CHECK")
print("="*60)
print(f"✅ Module imported")
print(f"🖥️  Device: {device}")
print(f"📊 EXACT threshold: N ≤ {EXACT_THRESHOLD:,}")
print(f"📊 STOCHASTIC: K = min({K_MULTIPLIER}×N, {K_MAX:,})")
if torch.cuda.is_available():
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"💾 VRAM: {torch.cuda.get_device_properties(0).total_memory/1e9:.1f} GB")
else:
    print("⚠️  WARNING: No GPU detected. Runtime will be slow.")
print(f"📁 Output dir: {BASE_DIR}")
print("="*60)

log_message(f"Environment ready. Device: {device}")

ENVIRONMENT CHECK
✅ Module imported
🖥️  Device: cuda
📊 EXACT threshold: N ≤ 10,000
📊 STOCHASTIC: K = min(100×N, 2,000,000)
🎮 GPU: Tesla T4
💾 VRAM: 15.8 GB
📁 Output dir: /content/drive/MyDrive/cgt_tree_runs/


---
## 🔬 Protocol E1: Distortion vs Dimension

**Objective:** Measure embedding dimension required for <10% distortion

In [ ]:
# @title 4️⃣ E1 — EXACT Mode (Validation)
# Purpose: Sanity check, confirm code works
# Tree: T_{3,7} = 3,280 nodes
# Time: ~10 min

STEP_NAME = "E1_exact"

if is_completed(STEP_NAME):
    print(f"⏭️ Skipping {STEP_NAME} (already completed)")
    results_e1_exact = load_result(STEP_NAME)
else:
    log_message(f"Starting {STEP_NAME}")

    results_e1_exact = run_protocol_e1(
        b=3,
        d=7,
        dimensions=[4, 8, 16, 32],
        n_trials=3,
        n_epochs=500,
        seed=42,
        mode="exact",
        verbose=True
    )

    # Save to Drive
    save_result(STEP_NAME, results_e1_exact)
    save_progress(STEP_NAME)
    log_message(f"Completed {STEP_NAME}")

print("\n" + "="*60)
print("✅ E1 EXACT COMPLETE")
print("="*60)
for k, v in results_e1_exact.get("latex", {}).items():
    print(f"  {k}: {v}")

Protocol E1: Distortion vs Dimension
  Tree: T_{3,7} with 3,280 nodes
  Mode: EXACT
  Dimensions: [4, 8, 16, 32]
  Trials: 3, Epochs: 500
  Device: cuda

  Dimension 4:
    Euclidean:  0.2983 ± 0.0000
    Hyperbolic: 0.8415 ± 0.0003

  Dimension 8:
    Euclidean:  0.2131 ± 0.0001
    Hyperbolic: 0.7588 ± 0.0002

  Dimension 16:
    Euclidean:  0.1760 ± 0.0000
    Hyperbolic: 0.6383 ± 0.0003

  Dimension 32:
    Euclidean:  0.1571 ± 0.0001
    Hyperbolic: 0.4964 ± 0.0002
💾 Result saved: /content/drive/MyDrive/cgt_tree_runs//results/E1_exact.json
💾 Progress saved: E1_exact

✅ E1 EXACT COMPLETE
  RESULT_E1_DIM_EUCLIDEAN: >32
  RESULT_E1_DIM_HYPERBOLIC: >32
  RESULT_E1_RATIO: N/A
  RESULT_E1_STATUS: NOT CONFIRMED


In [ ]:
# @title 5️⃣ E1 — STOCHASTIC Mode (Production)
# Purpose: Main result for paper
# Tree: T_{3,12} = 265,720 nodes
# Time: ~25 min

STEP_NAME = "E1_stochastic"

if is_completed(STEP_NAME):
    print(f"⏭️ Skipping {STEP_NAME} (already completed)")
    results_e1_stochastic = load_result(STEP_NAME)
else:
    log_message(f"Starting {STEP_NAME}")

    results_e1_stochastic = run_protocol_e1(
        b=3,
        d=12,
        dimensions=[4, 8, 16, 32, 64],
        n_trials=5,
        n_epochs=300,
        seed=42,
        mode="stochastic",
        verbose=True
    )

    # Save to Drive
    save_result(STEP_NAME, results_e1_stochastic)
    save_progress(STEP_NAME)
    log_message(f"Completed {STEP_NAME}")

print("\n" + "="*60)
print("✅ E1 STOCHASTIC COMPLETE")
print("="*60)
for k, v in results_e1_stochastic.get("latex", {}).items():
    print(f"  {k}: {v}")

Protocol E1: Distortion vs Dimension
  Tree: T_{3,12} with 797,161 nodes
  Mode: STOCHASTIC
  Dimensions: [4, 8, 16, 32, 64]
  Trials: 5, Epochs: 300
  Device: cuda

  Dimension 4:


---
## 🔬 Protocol E2: Depth Scaling

**Objective:** Fit power-law D(d) = A·d^α to distortion curves

In [ ]:
# @title 6️⃣ E2 — STOCHASTIC Mode
# Purpose: Scaling analysis
# Depths: 5→13 (crosses exact/stochastic boundary)
# Time: ~30 min

STEP_NAME = "E2_stochastic"

if is_completed(STEP_NAME):
    print(f"⏭️ Skipping {STEP_NAME} (already completed)")
    results_e2 = load_result(STEP_NAME)
else:
    log_message(f"Starting {STEP_NAME}")

    results_e2 = run_protocol_e2(
        b=3,
        dim=32,
        depths=[5, 7, 9, 11, 13],
        n_trials=5,
        n_epochs=300,
        seed=42,
        mode="stochastic",
        verbose=True
    )

    # Save to Drive
    save_result(STEP_NAME, results_e2)
    save_progress(STEP_NAME)
    log_message(f"Completed {STEP_NAME}")

print("\n" + "="*60)
print("✅ E2 COMPLETE")
print("="*60)
for k, v in results_e2.get("latex", {}).items():
    print(f"  {k}: {v}")
print(f"\nPower-law exponents:")
print(f"  α_Euclidean  = {results_e2['fit']['alpha_euclidean']:.3f}")
print(f"  α_Hyperbolic = {results_e2['fit']['alpha_hyperbolic']:.3f}")

---
## 🔬 Protocol E3: Topological Integrity

**Objective:** Detect spurious 1-cycles (β₁) via persistent homology

In [ ]:
# @title 7️⃣ E3 — STOCHASTIC Mode
# Purpose: Topological verification
# Depths: 5,7,9 (limited by ripser complexity)
# Time: ~15 min

STEP_NAME = "E3_stochastic"

if is_completed(STEP_NAME):
    print(f"⏭️ Skipping {STEP_NAME} (already completed)")
    results_e3 = load_result(STEP_NAME)
else:
    log_message(f"Starting {STEP_NAME}")

    results_e3 = run_protocol_e3(
        b=3,
        depths=[5, 7, 9],
        dim=32,
        n_trials=3,
        n_epochs=300,
        persistence_threshold=0.1,
        seed=42,
        mode="stochastic",
        verbose=True
    )

    # Save to Drive
    save_result(STEP_NAME, results_e3)
    save_progress(STEP_NAME)
    log_message(f"Completed {STEP_NAME}")

print("\n" + "="*60)
print("✅ E3 COMPLETE")
print("="*60)
for k, v in results_e3.get("latex", {}).items():
    print(f"  {k}: {v}")

---
## 📊 Results Export

In [ ]:
# @title 8️⃣ Generate LaTeX Commands
print("="*70)
print("📝 LaTeX Commands — Copy to paper preamble")
print("="*70)

all_latex = {}
all_latex.update(results_e1_exact.get("latex", {}))
all_latex.update(results_e1_stochastic.get("latex", {}))
all_latex.update(results_e2.get("latex", {}))
all_latex.update(results_e3.get("latex", {}))

# Save LaTeX to file
latex_file = f"{BASE_DIR}/results/latex_commands.tex"
with open(latex_file, "w") as f:
    f.write("% === Auto-generated LaTeX commands ===\n")
    f.write(f"% Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}\n\n")

    f.write("% === E1 Results ===\n")
    for k, v in sorted(all_latex.items()):
        if "E1" in k:
            cmd = k.replace("_", "")
            f.write(f"\\newcommand{{\\{cmd}}}{{{v}}}\n")
            print(f"\\newcommand{{\\{cmd}}}{{{v}}}")

    f.write("\n% === E2 Results ===\n")
    print("\n% === E2 Results ===")
    for k, v in sorted(all_latex.items()):
        if "E2" in k:
            cmd = k.replace("_", "")
            f.write(f"\\newcommand{{\\{cmd}}}{{{v}}}\n")
            print(f"\\newcommand{{\\{cmd}}}{{{v}}}")

    f.write("\n% === E3 Results ===\n")
    print("\n% === E3 Results ===")
    for k, v in sorted(all_latex.items()):
        if "E3" in k or "B1" in k:
            cmd = k.replace("_", "")
            f.write(f"\\newcommand{{\\{cmd}}}{{{v}}}\n")
            print(f"\\newcommand{{\\{cmd}}}{{{v}}}")

print(f"\n💾 LaTeX saved to: {latex_file}")
log_message("LaTeX commands generated")

In [ ]:
# @title 9️⃣ Summary Report
print("="*70)
print("📊 EXECUTION SUMMARY")
print("="*70)

completed_steps = load_progress()
print(f"\n✅ Completed steps: {len(completed_steps)}/4")
for step in completed_steps:
    print(f"   • {step}")

print(f"\n📁 All outputs saved to: {BASE_DIR}")
print(f"\n📄 Files in results/:")
for f in os.listdir(f"{BASE_DIR}/results"):
    size = os.path.getsize(f"{BASE_DIR}/results/{f}")
    print(f"   • {f} ({size:,} bytes)")

log_message("Execution summary generated")

In [ ]:
# @title 🧹 Final Cleanup
clear_gpu()
print("✅ GPU memory released")
print("\n" + "="*60)
print("🎉 ALL PROTOCOLS COMPLETE")
print("="*60)
print(f"\n📁 Results available at: {BASE_DIR}")
log_message("All protocols complete. GPU released.")

In [ ]:
# @title 📦 (Optional) Zip Results for Download
import shutil

zip_name = f"{BASE_DIR}/results_tree_run"
shutil.make_archive(zip_name, 'zip', f"{BASE_DIR}/results")

print(f"✅ Results zipped: {zip_name}.zip")
print(f"\nTo download, run:")
print(f"  from google.colab import files")
print(f"  files.download('{zip_name}.zip')")

---

## 📄 Methods Note (for paper)

```
For trees with more than 10⁴ nodes, explicit construction of full
pairwise distance matrices becomes infeasible due to quadratic memory
requirements. To ensure scalability, we replace the deterministic stress
objective with an unbiased Monte Carlo estimator via random subsampling
of K = min(2×10⁶, 100N) node pairs per epoch. Tree distances are computed
on-the-fly via LCA traversal, avoiding materialization of the distance matrix.
```

---

## ⚠️ DO NOT RUN

- `mode="exact"` for `d > 8`
- `depths` containing values > 13 in E3
- `n_epochs > 500`
- `dims` containing 128 or 256

---

**Author:** Éric Gustavo Reis de Sena  
**License:** CC BY-NC-SA 4.0